In [ ]:
import os
import pandas as pd
import numpy as np

import plotly.graph_objects as go

In [ ]:
utils_folder = os.path.join("..", "..", "utils")

data_folder = os.path.join("..", "..", "data")
clean_data_folder = os.path.join(data_folder, "Clean Data")
metadata_folder = os.path.join(data_folder, "Metadata")
plot_folder = os.path.join(data_folder, "Plots", "Feltre")

sensor_folder = os.path.join(clean_data_folder, "sensors")

feltre_sqlites_folder =  'feltre_sqlites'

# First Part

In [ ]:
feltre_df = pd.read_excel(os.path.join(clean_data_folder, 'feltre.xlsx'))

In [ ]:
target_variables = {
    'ICC [1/mL]': 'ICC (1/mL)',
    'HNAC [1/mL]': 'HNAC (1/mL)', 
    'LNAC [1/mL]': 'LNAC (1/mL)',
    'HNAP [%]': 'HNAP (%)',
}

In [ ]:
input_variables = {
    'Pressione [atm]': 'Pressione (atm)',
    'TOCeq [mg/l]': 'TOCeq (mg/l)',
    'DOCeq [mg/l]': 'DOCeq (mg/l)',
    'Turbidity [FTU]': 'Turbidity (FTU)', 
    'Conductivity [uS/cm]': 'Conductivity (uS/cm)',
    'Temperature [°C]': 'Temperature (°C)',
    'pH': 'pH',
    'Free Chlorine [mg/l]': 'Free Chlorine (mg/l)',
    'nitrati': 'nitrati',
    'UV254': 'UV254',
}

In [ ]:
feltre_df.rename(
    columns=input_variables,
    inplace=True
)
feltre_df.rename(
    columns=target_variables,
    inplace=True
)

In [ ]:
datasets = {}

for target_variable in target_variables.values():
    datasets[target_variable] = feltre_df[['DateTime', target_variable] + list(input_variables.values())].copy()
    datasets[target_variable].set_index('DateTime', inplace=True)
    datasets[target_variable].sort_index(inplace=True)
    datasets[target_variable].dropna(inplace=True)    

In [ ]:
from sklearn.preprocessing import MinMaxScaler 

# We are going to extend the features of the input variables for each target variable
# -
# We are going to add:

scaler = MinMaxScaler()

lags_in_hours = 3
shifts_in_indexes = int(0.25 * 4 * lags_in_hours)
rolling_window_in_hours = 6
rolling_window = int(0.25 * 4 * rolling_window_in_hours)
polyn_degree = 2

lstm_datasets = {}

for target_variable, df in datasets.items():
    X, y = df[list(input_variables.values())].copy(), df[target_variable].copy()
    
    X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)
    
    # we are going to use the log1p of the target variable for the modelling to avoid instability
    y = np.log1p(y)
    
    # need to change the name of target variable to avoid the / character
    
    target_variable = target_variable.replace("/", "_")
    
    # do not use the extended features for the LSTM model
    lstm_datasets[target_variable] = X, y

In [ ]:
for target_variable, (X, y) in lstm_datasets.items():
    print(f"Target variable: {target_variable}")
    # print number of nan values in X
    print(f"Number of nan values in X: {X.isna().sum().sum()}")
    # print number of nan values in y
    print(f"Number of nan values in y: {y.isna().sum().sum()}")
    print("-"*100)
    
    

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, TimeSeriesSplit

In [ ]:
# since the LSTM model takes as input a tensor of shape (num_samples, time_steps, n_features)
# we need to convert the pandas dataframe into a numpy array of shape (num_samples, time_steps, n_features)
# each sample is a sequence of window_size time steps, containing the features and the target variable
def create_sequences(X_df, y_df, window_size):
    """
    Converts Pandas DataFrames into overlapping sequences for LSTM input.
    
    Returns:
        X_seq: NumPy array of shape (num_samples - window_size, window_size, n_features)
        y_seq: NumPy array of shape (num_samples - window_size, 1) with the last target value of each window
        y_timestamps: List of timestamps corresponding to the predictions.
    """
    timesteps = X_df.index
    
    X_values = X_df.to_numpy()
    y_values = y_df.to_numpy()
    
    X_seq, y_seq, y_timestamps = [], [], []
    
    # Create sequences for X and corresponding y for only the last value of each window
    for i in range(len(X_values) - window_size):
        X_seq.append(X_values[i : i + window_size])  # Input sequence
        y_seq.append(y_values[i + window_size - 1])  # Only the last value in the target window
        y_timestamps.append(timesteps[i + window_size - 1])  # Timestamp for the last timestep
        
    return np.array(X_seq), np.array(y_seq), np.array(y_timestamps)


In [ ]:
window_size = 4

In [ ]:
X, y = lstm_datasets['HNAC (1_mL)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


X_train_seq, y_train_seq, timestamp_train = create_sequences(X_train, y_train, window_size)
X_test_seq, y_test_seq, timestamp_test = create_sequences(X_test, y_test, window_size)

bi_lstm_model = Sequential()
bi_lstm_model.add(Bidirectional(LSTM(units=64, return_sequences=True, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2]), seed=42)))
bi_lstm_model.add(Dropout(0.3))
bi_lstm_model.add(Bidirectional(LSTM(units=32)))
bi_lstm_model.add(Dropout(0.3))
bi_lstm_model.add(Dense(1))
bi_lstm_model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=MeanSquaredError(),
    metrics=[RootMeanSquaredError()],
)

bi_gru_model = Sequential()
bi_gru_model.add(Bidirectional(GRU(units=64, return_sequences=True, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2]), seed=42)))
bi_gru_model.add(Dropout(0.3))
bi_gru_model.add(Bidirectional(GRU(units=32)))
bi_gru_model.add(Dropout(0.3))
bi_gru_model.add(Dense(1))
bi_gru_model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=MeanSquaredError(),
    metrics=[RootMeanSquaredError()],
)

models = [bi_lstm_model, bi_gru_model]

In [ ]:
X_train_seq.shape, y_train_seq.shape

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Ensure that the validation data is provided in the correct format


for model in models:
    _ = model.fit(X_train_seq, y_train_seq, epochs=50, callbacks=[early_stopping], validation_split=0.2, batch_size=32)

In [ ]:
fig = go.Figure()

for model in models:
    

# Warm-up the model
    warm_up_pred = model.predict(X_train_seq[-window_size - 1:])
    warm_up_pred = np.squeeze(warm_up_pred)

    y_pred = model.predict(X_test_seq)
    y_pred = np.squeeze(y_pred)

    # concatenate the warm-up predictions with the test predictions
    y_pred = np.concatenate([warm_up_pred, y_pred])
    
    
    fig.add_trace(go.Scatter
    (
        x=timestamp_train,
        y=np.expm1(y_train), 
        mode='lines',
        name='True',
        line=dict(color='blue'),
        showlegend=False
    ))

    fig.add_trace(go.Scatter
    (
        x=timestamp_test,
        y=np.expm1(y_test),
        mode='lines',
        name='True',
        line=dict(color='blue'),
        showlegend=False
    ))

    fig.add_trace(go.Scatter
    (
        x=timestamp_test,
        y=np.expm1(y_pred),
        mode='lines',
        name=f'{model.name}',
    ))

target_variable_name = f"{target_variable.replace('_', '/')}"

# fig.update_yaxes(type="log")
fig.update_layout(
    xaxis_title="Time",
    yaxis_title=target_variable_name,
    margin=dict(l=0, r=10, t=30, b=0),
    font=dict(
        size=14,
    ),
)

# put the legend at the top
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter
(
    x=timestamp_train,
    y=np.expm1(y_train), 
    mode='lines',
    name='True',
    line=dict(color='blue'),
    showlegend=False
))

fig.add_trace(go.Scatter
(
    x=timestamp_test,
    y=np.expm1(y_test),
    mode='lines',
    name='True',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter
(
    x=timestamp_test,
    y=np.expm1(y_pred),
    mode='lines',
    name='Predicted',
    line=dict(color='red')
))

target_variable_name = f"{target_variable.replace('_', '/')}"

# fig.update_yaxes(type="log")
fig.update_layout(
    title={
        'text': f"{target_variable_name}",
        'y':0.98,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title="Time",
    yaxis_title=target_variable_name,
    margin=dict(l=0, r=10, t=30, b=0),
    font=dict(
        size=14,
    ),
)

# put the legend at the top
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.show()

In [ ]:
print(X_train_seq.shape, y_train_seq.shape)
print(X_test_seq.shape, y_test_seq.shape)


In [ ]:
# remove the first window_size samples of y_test
y_test = y_test[4:]

In [ ]:
y_test.shape

In [ ]:
y_test_pred.shape

In [ ]:
import datetime

In [ ]:
class WeightsLogger(tf.keras.callbacks.Callback):
    def __init__(self, log_dir):
        super(WeightsLogger, self).__init__()
        self.file_writer = tf.summary.create_file_writer(log_dir)

    def on_epoch_end(self, epoch, logs=None):
        with self.file_writer.as_default():
            for layer in self.model.layers:
                if len(layer.weights) > 0:  # Check if the layer has weights
                    for i, weight in enumerate(layer.weights):
                        tf.summary.histogram(f"{layer.name}/weight_{i}", weight, step=epoch)
        self.file_writer.flush()

In [ ]:
class GradientsLogger(tf.keras.callbacks.Callback):
    def __init__(self, log_dir):
        super(GradientsLogger, self).__init__()
        self.file_writer = tf.summary.create_file_writer(log_dir)

    def on_epoch_end(self, epoch, logs=None):
        X_train_seq, y_train_seq = self.model._training_data  # Assuming training data is accessible

        with tf.GradientTape() as tape:
            predictions = self.model(X_train_seq, training=True)
            loss = self.model.compiled_loss(y_train_seq, predictions)  # Compute loss
        
        gradients = tape.gradient(loss, self.model.trainable_variables)  # Compute gradients

        with self.file_writer.as_default():
            for i, grad in enumerate(gradients):
                if grad is not None:  # Some layers might not have gradients
                    tf.summary.histogram(f"gradients/var_{i}", grad, step=epoch)
        
        self.file_writer.flush()

In [ ]:
bi_lstm_model = Sequential()
bi_lstm_model.add(Input(shape=(window_size, X.shape[-1])))
bi_lstm_model.add(LSTM(units=n_units_1, return_sequences=True, seed=42))
bi_lstm_model.add(Dropout(dropout_1))
bi_lstm_model.add(LSTM(units=n_units_2, return_sequences=True, seed=42))
bi_lstm_model.add(Dropout(dropout_2))
bi_lstm_model.add(LSTM(units=n_units_3, seed=42))
bi_lstm_model.add(Dropout(dropout_3))
bi_lstm_model.add(Dense(n_neurons))
bi_lstm_model.add(Dense(1))
bi_lstm_model.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss=MeanSquaredError(),
    metrics=[RootMeanSquaredError()],
)

bi_lstm_model.summary()


log_dir = "lstm_logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")


early_stopping = EarlyStopping(monitor='root_mean_squared_error', patience=40, restore_best_weights=True)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True)
weights_logger = WeightsLogger(log_dir)
gradient_logger = GradientsLogger(log_dir)

bi_lstm_model._training_data = (X_train_seq, y_train_seq)

history = bi_lstm_model.fit(
    X_train_seq,
    y_train_seq,
    epochs=100,
    callbacks=[
        early_stopping,
        tensorboard_callback,
        weights_logger,
        gradient_logger
    ], 
    batch_size=batch_size,
)

In [ ]:
y_pred = bi_lstm_model.predict(X_test_seq)
y_pred = np.squeeze(y_pred)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir lstm_logs/fit

# Second Part

In [ ]:
def create_sequences(X_df, y_df, window_size):
    """
    Converts Pandas DataFrames into overlapping sequences for LSTM input.
    
    Returns:
        X_seq: NumPy array of shape (num_samples - window_size, window_size, n_features)
        y_seq: NumPy array of shape (num_samples - window_size, 1) with the last target value of each window
        y_timestamps: List of timestamps corresponding to the predictions.
    """
    timesteps = X_df.index
    
    X_values = X_df.to_numpy()
    y_values = y_df.to_numpy()
    
    X_seq, y_seq, y_timestamps = [], [], []
    
    # Create sequences for X and corresponding y for only the last value of each window
    for i in range(len(X_values) - window_size):
        X_seq.append(X_values[i : i + window_size])  # Input sequence
        y_seq.append(y_values[i + window_size - 1])  # Only the last value in the target window
        y_timestamps.append(timesteps[i + window_size - 1])  # Timestamp for the last timestep
        
    return np.array(X_seq), np.array(y_seq), np.array(y_timestamps)

In [ ]:
seed = 42

In [ ]:
second_part_df = pd.read_excel(os.path.join(clean_data_folder, 'Feltre', 'second_part.xlsx'))

In [ ]:
target_variables = {
    'ICC [1/mL]': 'ICC (1/mL)',
    'HNAC [1/mL]': 'HNAC (1/mL)', 
    'LNAC [1/mL]': 'LNAC (1/mL)',
    'HNAP [%]': 'HNAP (%)',
}

In [ ]:
input_variables = {
    'Pressione [atm]': 'Pressione (atm)',
    'TOCeq [mg/l]': 'TOCeq (mg/l)',
    'DOCeq [mg/l]': 'DOCeq (mg/l)',
    'Turbidity [FTU]': 'Turbidity (FTU)', 
    'Conductivity [uS/cm]': 'Conductivity (uS/cm)',
    'Temperature [°C]': 'Temperature (°C)',
    'pH': 'pH',
    'Free Chlorine [mg/l]': 'Free Chlorine (mg/l)',
    'Nitrate [mg/l]': 'Nitrate (mg/l)',
    'UV254 [1/m]': 'UV254 (1/m)',
}

In [ ]:
second_part_df.rename(
    columns=input_variables,
    inplace=True
)
second_part_df.rename(
    columns=target_variables,
    inplace=True
)

In [ ]:
datasets = {}

for target_variable in target_variables.values():
    datasets[target_variable] = second_part_df[['DateTime', target_variable] + list(input_variables.values())].copy()
    datasets[target_variable].set_index('DateTime', inplace=True)
    datasets[target_variable].sort_index(inplace=True)
    datasets[target_variable].dropna(inplace=True)    

In [ ]:
from sklearn.preprocessing import MinMaxScaler 

# We are going to extend the features of the input variables for each target variable
# -
# We are going to add:

scaler = MinMaxScaler()

lags_in_hours = 3
shifts_in_indexes = int(0.25 * 4 * lags_in_hours)
rolling_window_in_hours = 6
rolling_window = int(0.25 * 4 * rolling_window_in_hours)
polyn_degree = 2

ds = datasets.copy()
lstm_datasets = {}

for target_variable, df in datasets.items():
    ds[target_variable] = df[list(input_variables.values())].copy(), df[target_variable].copy()
    
    X = ds[target_variable][0]
    X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)
    
    # uncomment based on the dataset you want to use
    # X_extended = extend_features(X, lags_in_hours, rolling_window, polyn_degree)
    X_extended = X
    
    y = ds[target_variable][1]
    
    # we are going to use the log1p of the target variable for the modelling to avoid instability
    y = np.log1p(y)
    
    # need to change the name of target variable to avoid the / character
    ds.pop(target_variable)
    
    target_variable = target_variable.replace("/", "_")
    
    ds[target_variable] = X_extended, y
    
    # do not use the extended features for the LSTM model
    lstm_datasets[target_variable] = X, y
    
datasets = ds

In [ ]:
for target_variable, (X, y) in datasets.items():
    print(f"Target variable: {target_variable}")
    # print number of nan values in X
    print(f"Number of nan values in X: {X.isna().sum().sum()}")
    # print number of nan values in y
    print(f"Number of nan values in y: {y.isna().sum().sum()}")
    print("-"*100)
    
    

In [ ]:
predictions = {
    'LSTM' : {},
    'XGBoost': {},
    'LGBM': {},
    # 'QRNN': {},
    # 'GRU': {},
    # 'BI_LSTM': {}
}

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Dropout, Input, GRU, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
for target_variable, (X, y) in lstm_datasets.items():
    
    # modify based on the target variable to plot
    if target_variable != 'ICC (1_mL)':
        continue
    
    # ==== LSTM ====
    
    predictions['LSTM'][target_variable] = {}
    
    window_size = 12
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=seed)
    
    X_train_seq, y_train_seq, timesteps_train = create_sequences(X_train, y_train, window_size)
    X_test_seq, y_test_seq, timesteps_test = create_sequences(X_test, y_test, window_size)
    

    n_units_1 = 16
    n_neurons = 16
    learning_rate = 0.0001
    dropout_1 = 0.1
    batch_size = 12
    # n_units_2 = lstm_studies[target_variable].best_trial.params["n_units_2"]
    # dropout_2 = lstm_studies[target_variable].best_trial.params["dropout_2"]
    
    # fit the model 50 times to get a better estimate of the predictions and the uncertainty
    n_iterations = 1
    
    y_pred_list = []
    
    n_units_1 = 16
    n_neurons = 16
    learning_rate = 0.0001
    
    for _ in range(n_iterations):
        
        model = Sequential()
        model.add(Input(shape=(window_size, X_train_seq.shape[-1])))
        model.add(LSTM(units=n_units_1, return_sequences=False, seed=seed))
        # model.add(Dropout(dropout_1, seed=seed))
        # model.add(LSTM(units=n_units_2, return_sequences=False, seed=seed))
        # model.add(Dropout(dropout_2, seed=seed))
        model.add(Dense(n_neurons))
        model.add(Dense(1))
        model.compile(
            optimizer=Adam(learning_rate=learning_rate, epsilon=0.01),
            loss=MeanSquaredError(),
            metrics=[RootMeanSquaredError()],
        )
        
        early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

        history = model.fit(X_train_seq, y_train_seq, epochs=500, callbacks=[early_stopping], verbose=0, batch_size=batch_size, validation_split=0.3)
        
        # Warm-up the model
        warm_up_pred = model.predict(X_train_seq[-window_size - 1:])
        warm_up_pred = np.squeeze(warm_up_pred)
        
        y_pred = model.predict(X_test_seq)
        y_pred = np.squeeze(y_pred)

        # concatenate the warm-up predictions with the test predictions
        y_pred = np.concatenate([warm_up_pred, y_pred])
        
        y_pred_list.append(y_pred)
    
    # get a timesteps_test as a one-dimensional array with no duplicates
    timesteps_test = np.unique(timesteps_test)
    timesteps_train = np.unique(timesteps_train)

    predictions['LSTM'][target_variable]["timesteps_test"] = timesteps_test
    predictions['LSTM'][target_variable]["timesteps_train"] = timesteps_train
    predictions['LSTM'][target_variable]["y_test"] = y_test
    predictions['LSTM'][target_variable]["y_train"] = y_train
    
    mean_pred = np.mean(y_pred_list, axis=0)
    std_pred = np.std(y_pred_list, axis=0)
    
    predictions['LSTM'][target_variable]["mean_pred"] = mean_pred
    predictions['LSTM'][target_variable]["std_pred"] = std_pred

In [ ]:
fig = go.Figure()

# Add training and validation loss traces
fig.add_trace(
    go.Scatter(
        y=history.history['loss'][5:],
        name='Training Loss',
        line=dict(color='blue')
    )
)

fig.add_trace(
    go.Scatter(
        y=history.history['val_loss'][5:],
        name='Validation Loss',
        line=dict(color='red')
    )
)

# Update layout
fig.update_layout(
    title='Model Loss During Training',
    xaxis_title='Epoch',
    yaxis_title='Loss',
    showlegend=True
)

fig.show()

In [ ]:
fig = go.Figure()

# Add training and validation loss traces
fig.add_trace(
    go.Scatter(
        y=history.history['root_mean_squared_error'],
        name='Training Loss',
        line=dict(color='blue')
    )
)

fig.add_trace(
    go.Scatter(
        y=history.history['val_root_mean_squared_error'],
        name='Validation Loss',
        line=dict(color='red')
    )
)

# Update layout
fig.update_layout(
    title='RMSE Training and Validation',
    xaxis_title='Epoch',
    yaxis_title='RMSE',
    showlegend=True
)

fig.show()

In [ ]:
# LSTM PLOTS

for target_variable in lstm_datasets.keys():
    
    # modify based on the target variable to plot
    if target_variable != 'ICC (1_mL)':
        continue
    
    timesteps_test = predictions['LSTM'][target_variable]["timesteps_test"]
    timesteps_train = predictions['LSTM'][target_variable]["timesteps_train"]
    y_train = predictions['LSTM'][target_variable]["y_train"]
    y_test = predictions['LSTM'][target_variable]["y_test"]
    
    
    y_pred_lstm = predictions['LSTM'][target_variable]["mean_pred"]
    std_pred_lstm = predictions['LSTM'][target_variable]["std_pred"]    
    
    # y_pred_gru = predictions['GRU'][target_variable]["mean_pred"]
    # std_pred_gru = predictions['GRU'][target_variable]["std_pred"]
    
    # y_pred_bi_lstm = predictions['BI_LSTM'][target_variable]["mean_pred"]
    # std_pred_bi_lstm = predictions['BI_LSTM'][target_variable]["std_pred"]
    
    
    fig = go.Figure()
    fig.add_trace(go.Scatter
    (
        x=timesteps_train,
        y=np.expm1(y_train), 
        mode='lines',
        name='True',
        line=dict(color='blue'),
        showlegend=False
    ))
    
    fig.add_trace(go.Scatter
    (
        x=timesteps_test,
        y=np.expm1(y_test),
        mode='lines',
        name='True',
        line=dict(color='blue')
    ))
    
    fig.add_trace(go.Scatter
    (
        x=timesteps_test,
        y=np.expm1(y_pred_lstm),
        mode='lines',
        name='LSTM',
        line=dict(color='red')
    ))
    
    fig.add_trace(go.Scatter(
        name='Upper Bound',
        x=timesteps_test,
        y=np.expm1(y_pred_lstm + 1.96 * std_pred_lstm),
        mode='lines',
        line=dict(width=0),
        showlegend=False
    ))
    
    
    
    fig.add_trace(go.Scatter(
        name='Lower Bound',
        x=timesteps_test,
        y=np.expm1(y_pred_lstm - 1.96 * std_pred_lstm),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(255, 102, 102, 0.3)',  # light red color
        fill='tonexty',
        showlegend=False
    ))

    target_variable_name = f"{target_variable.replace('_', '/')}"
    
    
    fig.update_layout(
        title={
            'text': f"{target_variable_name}",
            'y':0.98,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis_title="Time",
        yaxis_title=target_variable_name,
        margin=dict(l=0, r=10, t=30, b=0),
        font=dict(
            size=14,
        ),
    )
    
    # put the legend at the top
    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ))
    
    fig.show()
    